In [ ]:
import folium
import json
from shapely.geometry import shape, Polygon

# Load GeoJSON data
with open('data/tracts_with_categories.geojson') as f:
    data = json.load(f)

with open('data/Los Angeles Neighborhood Map.geojson') as f:
    neighborhood_data = json.load(f)

basemap='https://{s}.basemaps.cartocdn.com/rastertiles/voyager/{z}/{x}/{y}{r}.png'

m = folium.Map(location=[34, -118],
               zoom_start=9,
               tiles=basemap,
               attr='My Data Attribution',
               tooltip='geojson'
              )

# Filter the data GeoJSON based on the 'HH' category
hh_data = {
    'type': 'FeatureCollection',
    'features': [f for f in data['features'] if f['properties']['category'] == 'HH']
}

# Define a function to style each feature in the filtered GeoJSON data based on its 'category' property
def style_function(feature):
    return {'fillColor': '#c50e0e', 'color': '#c50e0e', 'weight': 0, 'fillOpacity': 0.7}

# Define a new style function for the neighborhood data
def neighborhood_style_function(feature):
    return {'color': 'black', 'weight': 1}

# Add the filtered GeoJSON data to the map with the style function applied to each feature
folium.GeoJson(hh_data, style_function=style_function).add_to(m)

# Calculate the total area of the neighborhoods
total_area = sum([shape(feature['geometry']).area for feature in neighborhood_data['features']])

# Loop over the neighborhoods and check if they intersect with the HH locations
neighborhood_overlap = []
for neighborhood_feature in neighborhood_data['features']:
    neighborhood_shape = shape(neighborhood_feature['geometry'])
    # Use the buffer method to fix the geometry
    neighborhood_shape = neighborhood_shape.buffer(0)
    overlap = 0
    for hh_feature in hh_data['features']:
        hh_shape = shape(hh_feature['geometry'])
        # Use the buffer method to fix the geometry
        hh_shape = hh_shape.buffer(0)
        if neighborhood_shape.intersects(hh_shape):
            overlap += neighborhood_shape.intersection(hh_shape).area
    if overlap > 0:
        percentage = overlap / neighborhood_shape.area * 100
        neighborhood_feature['properties']['overlap_percentage'] = round(percentage, 2)
        neighborhood_overlap.append((neighborhood_feature['properties']['name'], round(percentage, 2)))
        folium.GeoJson(neighborhood_feature, style_function=neighborhood_style_function).add_to(m)

# Sort the neighborhoods by overlap percentage and print them out
sorted_neighborhood_overlap = sorted(neighborhood_overlap, key=lambda x: x[1], reverse=True)
for neighborhood, overlap in sorted_neighborhood_overlap:
    print(neighborhood, 'has an overlap percentage of', overlap)

# Display the map
m
